# yes24

- 책제목, 가격(정수), 상세페이지url, 썸네일이미지url
- 썸네일 이미지 다운로드 --> 상품 아이디 명으로 다운로드 받기


In [1]:
import requests
from bs4 import BeautifulSoup
import urllib
import urllib.parse as urlparser
import os

In [ ]:
# yes24 는 url encoding 2byte 체제...
# utf-8 인코딩이 아니다

# euc-kr, cp949, ms949.. 으로 인코딩 해야 한다.

In [2]:
"""
http://www.yes24.com/searchcorner/Search
?
keywordAd=&
keyword=&
domain=ALL&
qdomain=%C0%FC%C3%BC&
Wcode=001_005&
query=%C6%C4%C0%CC%BD%E3

"""
None

In [3]:
url_base = 'http://www.yes24.com/searchcorner/Search?keywordAd=&keyword=&domain=ALL&qdomain=%C0%FC%C3%BC&Wcode=001_005&query={search}'

In [14]:
keyword = '파이썬'

search = urlparser.quote(keyword, encoding='cp949') ## 인코딩!!
search


'%C6%C4%C0%CC%BD%E3'

In [15]:
url = url_base.format(search=search)
url

'http://www.yes24.com/searchcorner/Search?keywordAd=&keyword=&domain=ALL&qdomain=%C0%FC%C3%BC&Wcode=001_005&query=%C6%C4%C0%CC%BD%E3'

In [16]:
response = requests.get(url)
response

<Response [200]>

In [17]:
dom = BeautifulSoup(response.text, 'html.parser')

In [18]:
rr = dom.select_one("#schMid_wrap > div.goods_list_wrap.mgt30")
# rr = dom.select_one("#schMid_wrap")
len(rr)

15

In [19]:
book_list = rr.select("div.goodsList.goodsList_list table tr:nth-child(even)")
len(book_list)

20

In [21]:
"""
[{'thumbnail_url': 'http://image.yes24.com/Goods/74419916/L',
  'book_title': 'Do it! 점프 투 파이썬',
  'book_url': 'http://www.yes24.com/Product/Goods/74419916?OzSrank=1',
  'price': 16920},
 {'thumbnail_url': 'http://image.yes24.com/Goods/74269975/L',
  'book_title': '혼자 공부하는 파이썬',
  'book_url': 'http://www.yes24.com/Product/Goods/74269975?OzSrank=2',
  'price': 16200},
 {'thumbnail_url': 'http://image.yes24.com/Goods/96024871/L',
  'book_title': '혼자 공부하는 머신러닝+딥러닝',
  'book_url': 'http://www.yes24.com/Product/Goods/96024871?OzSrank=3',
  'price': 23400},
  ...
]
"""
None

In [28]:
result = []

for book_item in book_list:
    # 썸네일 url
    img = book_item.select_one("td.goods_img > a > img")
    img_src = img.attrs['src'].strip()  
    # 책제목
    book_title = book_item.select_one("td.goods_infogrp > p.goods_name > a").text.strip()
    # 책 url
    book_url = book_item.select_one("td.goods_infogrp > p.goods_name > a").attrs['href']
    book_url = "http://www.yes24.com" + book_url 
    # 책 가격
    price = int(book_item.select_one("td.goods_infogrp > .goods_price > em").text.strip().replace(",", ""))     
    
    result.append({
        "thumbnail_url" : img_src,
        "book_title" : book_title,
        "book_url" : book_url,
        "price" : price
    })

result

[{'thumbnail_url': 'http://image.yes24.com/Goods/74419916/L',
  'book_title': 'Do it! 점프 투 파이썬',
  'book_url': 'http://www.yes24.com/Product/Goods/74419916?OzSrank=1',
  'price': 16920},
 {'thumbnail_url': 'http://image.yes24.com/Goods/74269975/L',
  'book_title': '혼자 공부하는 파이썬',
  'book_url': 'http://www.yes24.com/Product/Goods/74269975?OzSrank=2',
  'price': 16200},
 {'thumbnail_url': 'http://image.yes24.com/Goods/96024871/L',
  'book_title': '혼자 공부하는 머신러닝+딥러닝',
  'book_url': 'http://www.yes24.com/Product/Goods/96024871?OzSrank=3',
  'price': 23400},
 {'thumbnail_url': 'http://image.yes24.com/Goods/91433923/L',
  'book_title': '이것이 취업을 위한 코딩 테스트다 with 파이썬',
  'book_url': 'http://www.yes24.com/Product/Goods/91433923?OzSrank=4',
  'price': 30600},
 {'thumbnail_url': 'http://image.yes24.com/Goods/98135876/L',
  'book_title': '파이썬을 이용한 비트코인 자동매매',
  'book_url': 'http://www.yes24.com/Product/Goods/98135876?OzSrank=5',
  'price': 24300},
 {'thumbnail_url': 'http://image.yes24.com/Goods/9057

## 썸네일 이미지 다운로드

In [29]:
# ./yes24 폴더 만들고

# 위 검색결과에서 썸네일 이미지만 위 폴더에 다운로드 하기

# 저장 파일 형태는
# yes24_파이썬_1_74419916.jpg
# yes24_파이썬_2_74269975.jpg
# yes24_파이썬_3_96024871.jpg
# yes24_파이썬_4_91433923.jpg
#  ...


In [30]:
basepath = r'D:\DevRoot\DataSet'

In [32]:
dirpath = os.path.join(basepath, 'download', 'yes24')
if not os.path.exists(dirpath):
    os.mkdir(dirpath)
    print(dirpath, '디렉토리 생성')

D:\DevRoot\DataSet\download\yes24 디렉토리 생성


In [39]:
i = 1
for item in result:
    url = item.get('thumbnail_url')
    book_id = urlparser.urlsplit(url).path.split('/')[2]
    download_name = "yes24_{keyword}_{index}_{book_id}.jpg".format(keyword=keyword, index=i, book_id=book_id)
    print("다운로드", url, '->', download_name)
    urllib.request.urlretrieve(url, os.path.join(dirpath, download_name))
    i += 1

다운로드 http://image.yes24.com/Goods/74419916/L -> yes24_파이썬_1_74419916.jpg
다운로드 http://image.yes24.com/Goods/74269975/L -> yes24_파이썬_2_74269975.jpg
다운로드 http://image.yes24.com/Goods/96024871/L -> yes24_파이썬_3_96024871.jpg
다운로드 http://image.yes24.com/Goods/91433923/L -> yes24_파이썬_4_91433923.jpg
다운로드 http://image.yes24.com/Goods/98135876/L -> yes24_파이썬_5_98135876.jpg
다운로드 http://image.yes24.com/Goods/90578506/L -> yes24_파이썬_6_90578506.jpg
다운로드 http://image.yes24.com/Goods/101818336/L -> yes24_파이썬_7_101818336.jpg
다운로드 http://image.yes24.com/Goods/91437485/L -> yes24_파이썬_8_91437485.jpg
다운로드 http://image.yes24.com/Goods/91446225/L -> yes24_파이썬_9_91446225.jpg
다운로드 http://image.yes24.com/Goods/91084402/L -> yes24_파이썬_10_91084402.jpg
다운로드 http://image.yes24.com/Goods/87044746/L -> yes24_파이썬_11_87044746.jpg
다운로드 http://image.yes24.com/Goods/101876329/L -> yes24_파이썬_12_101876329.jpg
다운로드 http://image.yes24.com/Goods/101861210/L -> yes24_파이썬_13_101861210.jpg
다운로드 http://image.yes24.com/Goods/1018913